In [2]:
datalist = list()
with open("inst6502_comp.txt", "r") as f:
    for line in f:
        data = line.strip().split('\t')
        datalist.extend([i.split(" ",1) for i in data])

datatypes = list(set([i[1] for i in datalist]))
print(datatypes)
#print(len(datatypes))
with open("../debug_itab.z80", "w") as f:
    f.write("inst6502_opcode_width_table:\n")
    for idx, i in enumerate(datatypes):
        size = 0
        if i in ("", "impl", "A"):
            size = 1
        elif i in ("zpg", "rel", "X,ind", "ind,Y", "#", "zpg,X", "zpg,Y"):
            size = 2
        elif i in ("abs", "abs,X", "abs,Y", "ind"):
            size = 3
        pass
        s = f".db {size}    ;{idx}: {i}\n"
        f.write(s)
    f.write("\n\n;Note: 1=1byteOper, 2=2byteOper, -=1byteRelOper. Other chrs are literal\n")
    f.write("\ninst6502_disassembler:\n")
    for idx, i in enumerate(datatypes):
        if i == "":
            s = ""
        elif i == "impl":
            s = ""
        elif i == "A":
            s = "A"
        elif i == "#":
            s = "#1"
        elif i == "zpg":
            s = "1"
        elif i == "rel":
            s = "-"
        elif i == "zpg,X":
            s = "1,X"
        elif i == "zpg,Y":
            s = "1,Y"
        elif i == "X,ind":
            s = "(1,X)"
        elif i == "ind,Y":
            s = "(1),Y"
        elif i == "ind":
            s = "(2)"
        elif i == "abs":
            s = "2"
        elif i == "abs,X":
            s = "2,X"
        elif i == "abs,Y":
            s = "2,Y"
        size = len(s)+1
        f.write(f'.db {size},"{s}"  ;\n')
        


    f.write("\ninst6502_disassembly_table:\n")
    for idx, i in enumerate(datalist):
        s = f'.db "{i[0].strip()[:3]}", {datatypes.index(i[1])}     ;0x{idx:02X}\n'
        f.write(s)





['', 'zpg,X', 'ind', 'abs', 'abs,X', 'X,ind', 'A', 'zpg', 'rel', '#', 'impl', 'ind,Y', 'abs,Y', 'zpg,Y']


In [5]:
import hashlib, os, importlib
import toolkit
importlib.reload(toolkit)

def tobytes(indata:list):
    outlist = list()
    for dat in indata:
        if isinstance(dat,str):
            for s in dat:
                outlist.append(ord(s))
        elif isinstance(dat,(bytes, bytearray, list)):
            for b in dat:
                outlist.append(b)
        else:
            outlist.append(dat)
    return bytes(outlist)


filename = "nestest.nes"
filebase = filename.split('.')[0]
with open("nestest.nes","rb") as f:
    data = bytes(f.read())

print(data[:15])
isrom = False
def lo(num):
    return num & 0xFF
def hi(num):
    return (num >> 8) & 0xFF
if bytes(data[:4]) == bytes([ord('N'),ord('E'),ord('S'), 0x1A]):
    print("Is NES ROM")
    isrom = True
    prg_rom_count = int(data[4])
    chr_rom_count = int(data[5])
    print(f"PRG COUNT {prg_rom_count}, CHR COUNT {chr_rom_count}")
    hashdata = hashlib.sha256(data).digest()[:16]
    print(hashdata)
    datalen = len(data)
    convdata = tobytes(["CENROM", 0, filebase, hashdata, lo(datalen), hi(datalen), data])
    toolkit.export8xv(filebase, convdata)
    if os.path.exists(f"../bin/{filebase}.8xv"):
        os.remove(f"../bin/{filebase}.8xv")
    os.rename(f"{filebase}.8xv",f"../bin/{filebase}.8xv")
    a = (data[6] & 0xF0) >> 4
    b = (data[7] & 0xF0)
    print(f"Mapper ID: {a|b}")



if not isrom:
    print("Failed an NES ROM check.")




b'NES\x1a\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Is NES ROM
PRG COUNT 1, CHR COUNT 1
b'\xf6}U\xfdk<\xf0\xba\xd1\xcc\x85\xf1\xdf\rs\x9c'


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'nestest.8xv' -> '../bin/nestest.8xv'